In [1]:
#Functions for parsing columns of the spreadsheet on Santa Clara County School Meals
import re

def fcnparse_opendays(stringopendays):
    print(stringopendays)
    
def fcnparse_siteaddress(csvaddress):
    myoutput = nominatim.query(csvaddress+ ' California')
    if len(myoutput.toJSON())>0:
        myjson = myoutput.toJSON()[0]
    
        #print(myjson)
        #print(type(myjson))
        full_address = myjson['display_name']
        #print(full_address)
        #print(type(full_address))

        tmp = full_address.split(', ')
        #print(tmp)
       
        if tmp[0].isnumeric():
            address = tmp[0]+" "+tmp[1]
            city = tmp[-5:-4][0]
            state = tmp[-3:-2][0]
            zipcode = tmp[-2:-1][0]            
        elif tmp[1].isnumeric():
            address = tmp[1]+" "+tmp[2]
            city = tmp[-5:-4][0]
            state = tmp[-3:-2][0]
            zipcode = tmp[-2:-1][0]
        else:
            address = csvaddress+" ######CHECK ME######"
            city = "######CHECK ME######"
            zipcode = "######CHECK ME######"
            state = "California"
    else:
        full_address = ""
        address = csvaddress+"######CHECK ME######"
        city = "######CHECK ME######"
        zipcode = "######CHECK ME######"
        state = "California"
        
    return full_address, address, city, state, zipcode

def fcnparse_opendays(attributesOPENDAYS):
    return ["Monday","Tuesday","Thursday"]

def fcnparse_opentimes(attributesOPENTIMES):
    return "2:00 PM", "6:00 PM"

def fcnparse_mealtypes(attributesMEALTYPES):
    #print(attributesMEALTYPES)
    mealoptions = []
    if re.search('breakfast', attributesMEALTYPES, re.IGNORECASE):
        mealoptions = mealoptions + ["Breakfast", ]
    if re.search('lunch', attributesMEALTYPES, re.IGNORECASE):
        mealoptions = mealoptions + ["Lunch", ]  
    if re.search('dinner', attributesMEALTYPES, re.IGNORECASE):
        mealoptions = mealoptions + ["Dinner", ]   

    #print(mealoptions)
    return mealoptions


In [9]:
#Code to scrape Santa Clara County school meal sites
import requests
import csv
from OSMPythonTools.nominatim import Nominatim
nominatim = Nominatim()

post_url_services = 'https://api.airtable.com/v0/applQOkth8R2ns3qo/services'
post_url_organizations = 'https://api.airtable.com/v0/applQOkth8R2ns3qo/organizations'
post_url_address = 'https://api.airtable.com/v0/applQOkth8R2ns3qo/address'
post_url_schedule = 'https://api.airtable.com/v0/applQOkth8R2ns3qo/schedule'
post_url_phones = 'https://api.airtable.com/v0/applQOkth8R2ns3qo/phones'

post_headers = {
    'Authorization' : 'Bearer keyFaCMgt8zX71GnJ',
    'Content-Type': 'application/json'
}

csvfilename = open('/Users/Ashwin/Documents/BAC/20200919cfsj/Santa Clara Scrape 10_1 - Food School Meal Sites .csv')
csv_f = csv.reader(csvfilename)

#Skip header row
count = 1
for row in csv_f:
    if count == 1:
        break

#Loop over all rows and parse out the columns in each row
count = 1
for row in csv_f:
    #print(row)
    rowcount = 0
    for elem in row:
        print(rowcount, elem)
        rowcount = rowcount+1
    
    #Parse data for PHONES sheet: attributes.PHONENUMBER
    attributesPHONENUMBER = row[14]
    airtablePHONES = {"fields": {"number": attributesPHONENUMBER}}
    
    #Parse data for ORGANIZATIONS sheet: attributes.SITESCHOOLDISTRICT
    attributesSITESCHOOLDISTRICT = row[4]
    airtableORGANIZATIONS = {"fields": {"name": attributesSITESCHOOLDISTRICT}}

    #Parse data for SERVICES sheet: attributes.SITENAME, attributes.MEALTYPES, 
    #attributes.SITESTATUS, attributes.DESCRIPTION, attributes.EMAIL, attributes.WEBLINK
    attributesSITENAME = row[5]
    attributesMEALTYPES = fcnparse_mealtypes(row[10])
    attributesSITESTATUS = row[12]
    attributesDESCRIPTION = row[13]+" "+row[23]    
    attributesEMAIL = row[15]
    attributesWEBLINK = row[16]
    airtableSERVICES = {"fields": {"Name": attributesSITENAME, 
                                   "freefood_area": attributesMEALTYPES, 
                                   "active": attributesSITESTATUS,
                                   "Description": attributesDESCRIPTION,
                                   "email": attributesEMAIL,
                                   "url": attributesWEBLINK}}

    
    #Parse data for ADDRESS sheet: attributes.SITEADDRESS   
    attributesSITEADDRESS = row[6]
    #print(attributesSITEADDRESS)
    full_address, address, city, state, zipcode = fcnparse_siteaddress(attributesSITEADDRESS)
    #print(full_address)
    #print(count+1, address, city, state, zipcode, '\n')
    airtableADDRESS = {"fields": {"address_1": address,"city": city,"State": state,"Zip Code": zipcode}} 
        
    #Parse data for SCHEDULE sheet: attributes.OPENDAYS, attributes.OPENTIMES
    attributesOPENDAYS = row[7]
    attributesOPENTIMES = row[8]
    opendays = fcnparse_opendays(attributesOPENDAYS)
    opensat, closesat = fcnparse_opentimes(attributesOPENTIMES)
    airtableSCHEDULE = {"fields": {"weekday": opendays, "opens_at": opensat, "closes_at": closesat}} 
    
    #Upload to airtable
    print((requests.post(post_url_phones, headers = post_headers, json = airtablePHONES)).status_code)    
    ((requests.post(post_url_organizations, headers = post_headers, json = airtableORGANIZATIONS)).status_code)    
    print((requests.post(post_url_services, headers = post_headers, json = airtableSERVICES)).status_code)    
    ((requests.post(post_url_address, headers = post_headers, json = airtableADDRESS)).status_code)    
    ((requests.post(post_url_schedule, headers = post_headers, json = airtableSCHEDULE)).status_code)

    count = count + 1
    if count == 2:
        break    

0 0
1 1
2 1
3 1
4 Alum Rock Union SD
5 Painter/Sheppard MS
6 480 Rough & Ready Rd. San Jose 
 
 

7 Monday through Thursday


De lunes a jueves
8 11:00 a.m. to 1:00 p.m.
9 On-going through school June 9th
10 Lunch and Breakfast
11 School
12 Open
13 The school meal distribution program provides food for low-income families whose children depend on the lunch or breakfast they get from the cafeteria.
14 
15 
16 https://resources.finalsite.net/images/v1584407264/arusdorg/ctuat3itibpp3impkobc/FamilyLetter3-6-20.pdf
17 San Jose
18 5
19 1588782424000
20 FME
21 *updated 5.4.20 at 4:12 p.m.
22 EOC-FCMS-0000000001
23 Curbside pickup or walk through.
24 -13563497.92
25 4491488.531
200
200


In [27]:
#Prototyping code for parsing addresses

from OSMPythonTools.nominatim import Nominatim
nominatim = Nominatim()
#myoutput = nominatim.query('1680 Foley ave')
#print(myoutput.toJSON())
csvaddress = '592 Dunholme Way, san jose'
myoutput = nominatim.query(csvaddress)
print(myoutput)
print(myoutput.toJSON())
print(len(myoutput.toJSON()))
myjson = myoutput.toJSON()[0]
print(myjson)
print(type(myjson))
full_address = myjson['display_name']
print(full_address)
print(type(full_address))

tmp = full_address.split(', ')
print(tmp)
firstelem = tmp[0]
print(firstelem)
print(firstelem.isnumeric())
if tmp[0].isnumeric():
    address = tmp[0]+" "+tmp[1]
    city = tmp[-5:-4][0]
    state = tmp[-3:-2][0]
    zipcode = tmp[-2:-1][0]            
elif tmp[1].isnumeric():
    address = tmp[1]+" "+tmp[2]
    city = tmp[-5:-4][0]
    state = tmp[-3:-2][0]
    zipcode = tmp[-2:-1][0]
else:
    address = csvaddress+" ######CHECK ME######"
    city = "######CHECK ME######"
    zipcode = "######CHECK ME######"
    state = "California"

print(address)
print(city)
print(state)
print(zipcode)

[nominatim] downloading data: 592 Dunholme Way, san jose
[]
0


IndexError: list index out of range

In [87]:
if ~len([]):
    print("yup empty")

yup empty


In [123]:
#blah = '1680'
#blah.isdigit()
print(1!=2)

True


In [11]:
#Prototyping code for parsing meal types

import re

if re.search('mandye', 'Mandy Pande', re.IGNORECASE):
    print("Yes found")
    
test_tup1 = [1, 3, 5] 
test_tup2 = [4,]
  
# printing original tuples 
print("The original tuple 1 : " + str(test_tup1)) 
print("The original tuple 2 : " + str(test_tup2)) 
  
# Ways to concatenate tuples 
# using + operator 
res = test_tup1 + test_tup2 
  
# printing result 
print("The tuple after concatenation is : " + str(res))

The original tuple 1 : [1, 3, 5]
The original tuple 2 : [4]
The tuple after concatenation is : [1, 3, 5, 4]
